In [70]:
from faker import Faker
fake = Faker()
sents = fake.sentences(100)


In [81]:
from __future__ import print_function

import argparse
import time

import grpc

from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

import tensorflow as tf

# def run(host, port, model, signature_name):

host = 'localhost'
model = 'use'
signature_name = 'serve'
port = 8500
channel = grpc.insecure_channel('{host}:{port}'.format(host=host, port=port))
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

start = time.time()
for x in range(50):

    # Call classification model to make prediction
    request = predict_pb2.PredictRequest()
    request.model_spec.name = model
    # request.model_spec.signature_name = signature_name
    request.inputs['inputs'].CopyFrom(tf.make_tensor_proto(sents, dtype=tf.dtypes.string))
    # request.inputs['sepal_width'].CopyFrom(make_tensor_proto(3.2, shape=[1, 1]))
    # request.inputs['petal_length'].CopyFrom(make_tensor_proto(5.9, shape=[1, 1]))
    # request.inputs['petal_width'].CopyFrom(make_tensor_proto(2.3, shape=[1, 1]))

    result = stub.Predict(request,10.0)
    


    # Reference:
    # How to access nested values
    # https://stackoverflow.com/questions/44785847/how-to-retrieve-float-val-from-a-predictresponse-object
    # result.outputs[0]

    len(result.outputs['outputs'])
print(time.time() - start)


0.7407107353210449


In [82]:
import requests
DOCKER_HOST = 'localhost'

start = time.time()
for x in range(50):
    data = {
    "inputs":  sents,
    }

    records = []
    x = requests.post(f'http://{DOCKER_HOST}:8503/v1/models/use:predict', json=data).json()
    len(x['outputs'])
print(time.time() - start)


3.105710983276367


In [76]:
result.outputs['outputs'].float_val[0:10]

[-0.041076887398958206,
 0.0488143116235733,
 -0.053028929978609085,
 0.02703404799103737,
 0.047207556664943695,
 0.07293877750635147,
 0.020211076363921165,
 -0.05845915898680687,
 -0.061308715492486954,
 0.0014748353278264403]